# Differential cross section

In [2]:
%matplotlib inline

In [3]:
# %load preamb.py
import numpy as np
import pylab as plt
from matplotlib import rcParams
from scipy.optimize import curve_fit
import scipy.constants as co
import uncertainties as uc
import uncertainties.unumpy as un
from scipy.signal import argrelextrema as ext
from scipy.signal import savgol_filter as sav
import seaborn as sns
from scipy.special import erfc
from scipy.integrate import quad
import sys

sns.set(style='ticks', palette='Set2')
sns.despine()

fontsize_labels = 22    # size used in latex document
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['text.usetex'] = True
rcParams['figure.autolayout'] = True
rcParams['font.size'] = fontsize_labels
rcParams['axes.labelsize'] = fontsize_labels
rcParams['xtick.labelsize'] = fontsize_labels
rcParams['ytick.labelsize'] = fontsize_labels
rcParams['legend.fontsize'] = fontsize_labels
rcParams['figure.figsize'] = (2*6.2, 2*3.83)  # in inches; width corresponds to \textwidth in latex document (golden ratio)

plt.close("all")
show_fig = True
save_fig = False # see below
if not save_fig:
        rcParams['figure.figsize'] = (13, 8)
        fig_dir = "../figures/"
        npy_dir = "./data_npy/"


## Integration

Photon intensities at different angles. 
Integration is done numerically: sum over entries of histogram, 
multiplied by width of bin in energy domain (result of calibration_na). 
The differential cross section is approximated by the fraction of 
photons scattered at angle $\theta$ ($I(\theta)$) over the total incident 
intensity ($I_{tot}$), divided by the solid angle $\Delta \Omega$.

In [18]:
sums = []
theta = 75
y = np.load(npy_dir + 'na_rate_' + str(theta) + '.npy')
sums.append(np.sum(y))

na_cal = np.load(npy_dir + 'na_calibration.npy')
dE = na_cal[0]
integral = np.array(sums) * dE

'epsilon(energy)'
'mu(energy)'

'mu(energy)'

In [19]:
integral

array([0.15120726412252344+/-0.002002356430976444], dtype=object)

must further include: errors on counts for all measurements